--- Day 17: Reservoir Research ---

You arrive in the year 18. If it weren't for the coat you got in 1018, you would be very cold: the North Pole base hasn't even been constructed.

Rather, it hasn't been constructed yet. The Elves are making a little progress, but there's not a lot of liquid water in this climate, so they're getting very dehydrated. Maybe there's more underground?

You scan a two-dimensional vertical slice of the ground nearby and discover that it is mostly sand with veins of clay. The scan only provides data with a granularity of square meters, but it should be good enough to determine how much water is trapped there. In the scan, x represents the distance to the right, and y represents the distance down. There is also a spring of water near the surface at x=500, y=0. The scan identifies which square meters are clay (your puzzle input).

For example, suppose your scan shows the following veins of clay:
```
x=495, y=2..7
y=7, x=495..501
x=501, y=3..7
x=498, y=2..4
x=506, y=1..2
x=498, y=10..13
x=504, y=10..13
y=13, x=498..504
```
Rendering clay as `#`, sand as `.`, and the water spring as `+`, and with `x` increasing to the right and `y` increasing downward, this becomes:
```
   44444455555555
   99999900000000
   45678901234567
 0 ......+.......
 1 ............#.
 2 .#..#.......#.
 3 .#..#..#......
 4 .#..#..#......
 5 .#.....#......
 6 .#.....#......
 7 .#######......
 8 ..............
 9 ..............
10 ....#.....#...
11 ....#.....#...
12 ....#.....#...
13 ....#######...
```
The spring of water will produce water forever. Water can move through sand, but is blocked by clay. Water always moves down when possible, and spreads to the left and right otherwise, filling space that has clay on both sides and falling out otherwise.

For example, if five squares of water are created, they will flow downward until they reach the clay and settle there. Water that has come to rest is shown here as `~`, while sand through which water has passed (but which is now dry again) is shown as `|`:
```
......+.......
......|.....#.
.#..#.|.....#.
.#..#.|#......
.#..#.|#......
.#....|#......
.#~~~~~#......
.#######......
..............
..............
....#.....#...
....#.....#...
....#.....#...
....#######...
```
Two squares of water can't occupy the same location. If another five squares of water are created, they will settle on the first five, filling the clay reservoir a little more:
```
......+.......
......|.....#.
.#..#.|.....#.
.#..#.|#......
.#..#.|#......
.#~~~~~#......
.#~~~~~#......
.#######......
..............
..............
....#.....#...
....#.....#...
....#.....#...
....#######...
```
Water pressure does not apply in this scenario. If another four squares of water are created, they will stay on the right side of the barrier, and no water will reach the left side:
```
......+.......
......|.....#.
.#..#.|.....#.
.#..#~~#......
.#..#~~#......
.#~~~~~#......
.#~~~~~#......
.#######......
..............
..............
....#.....#...
....#.....#...
....#.....#...
....#######...
```
At this point, the top reservoir overflows. While water can reach the tiles above the surface of the water, it cannot settle there, and so the next five squares of water settle like this:
```
......+.......
......|.....#.
.#..#||||...#.
.#..#~~#|.....
.#..#~~#|.....
.#~~~~~#|.....
.#~~~~~#|.....
.#######|.....
........|.....
........|.....
....#...|.#...
....#...|.#...
....#~~~~~#...
....#######...
```
Note especially the leftmost |: the new squares of water can reach this tile, but cannot stop there. Instead, eventually, they all fall to the right and settle in the reservoir below.

After 10 more squares of water, the bottom reservoir is also full:
```
......+.......
......|.....#.
.#..#||||...#.
.#..#~~#|.....
.#..#~~#|.....
.#~~~~~#|.....
.#~~~~~#|.....
.#######|.....
........|.....
........|.....
....#~~~~~#...
....#~~~~~#...
....#~~~~~#...
....#######...
```
Finally, while there is nowhere left for the water to settle, it can reach a few more tiles before overflowing beyond the bottom of the scanned data:
```
......+.......    (line not counted: above minimum y value)
......|.....#.
.#..#||||...#.
.#..#~~#|.....
.#..#~~#|.....
.#~~~~~#|.....
.#~~~~~#|.....
.#######|.....
........|.....
...|||||||||..
...|#~~~~~#|..
...|#~~~~~#|..
...|#~~~~~#|..
...|#######|..
...|.......|..    (line not counted: below maximum y value)
...|.......|..    (line not counted: below maximum y value)
...|.......|..    (line not counted: below maximum y value)
```
How many tiles can be reached by the water? To prevent counting forever, ignore tiles with a y coordinate smaller than the smallest y coordinate in your scan data or larger than the largest one. Any x coordinate is valid. In this example, the lowest y coordinate given is 1, and the highest is 13, causing the water spring (in row 0) and the water falling off the bottom of the render (in rows 14 through infinity) to be ignored.

So, in the example above, counting both water at rest (~) and other sand tiles the water can hypothetically reach (|), the total number of tiles the water can reach is 57.

How many tiles can the water reach within the range of y values in your scan?


In [46]:
from collections import defaultdict, deque, Counter
import re

In [71]:
data = open('input.txt').read().splitlines()

testdata = '''x=495, y=2..7
y=7, x=495..501
x=501, y=3..7
x=498, y=2..4
x=506, y=1..2
x=498, y=10..13
x=504, y=10..13
y=13, x=498..504'''.splitlines()

testdata2 = '''x=495, y=2..7
y=7, x=495..501
y=3, x=500..501
x=501, y=4..7
x=498, y=2..4
x=506, y=1..2
x=498, y=10..13
x=504, y=10..13
y=13, x=498..504'''.splitlines()


In [72]:
UP = complex(0, -1)
DOWN = complex(0, 1)
LEFT = complex(-1, 0)
RIGHT = complex(1, 0)

class GroundScan(object):
    
    def __init__(self, data):
        self.state = self._parse(data)
        self.ymin = min(int(e.imag) for e in self.state.keys())
        self.ymax = max(int(e.imag) for e in self.state.keys())
        
    def _parse(self, data):
        clay = []
        for line in data:
            first, c1, cfrom, cto = re.search(r'^([xy])=([^,]+), [xy]=(-?[0-9]+)..(-?[0-9]+)$', line).groups()
            c1, cfrom, cto = map(int, [c1, cfrom, cto])
            y = range(cfrom, cto + 1)
            x = [c1] * len(y)
            if first == 'y':
                x, y = y, x
            clay += list(zip(x, y))
        layout = defaultdict(lambda: '.')
        for c in clay:
            layout[complex(*c)] = '#'
        return layout
    
    def addWater(self, source=complex(500, 0)):
        start = source + DOWN
 
        if self.state[start] != '.':
            return
        
        self.state[start] = '|'

        queue = deque()
        
        backstack = list()
        backstack.append(start)
        
        leftblock = None
        rightblock = None
        
        while len(backstack):
            #print('\n\n')
            poppedlast = False
            if len(queue) == 0:
                poppedlast = True
                #print('popped up')
                queue.append(backstack.pop())
                leftblock = None
                rightblock = None
            pos = queue.popleft()
            #print(queue)
            #print(backstack)
            ##print(f'1pos = {pos}')
            #print(str(g))
            if (pos + DOWN).imag <= self.ymax: # not off the bottom
                if self.state[pos + DOWN] == '.':
                    #print('added down')
                    self.state[pos + DOWN] = '|'
                    queue.append(pos + DOWN)
                    backstack.append(pos)
                    leftblock = None
                    rightblock = None
                    continue # don't try left or right until we find ourselves blocked below
            else:
                continue
            ###print(f'2pos = {pos}')
            if (self.state[pos + LEFT] == '.') and (self.state[pos + LEFT + DOWN] in '#~.') and (self.state[pos + DOWN] in '#|~'):
                if (not poppedlast) or (self.state[pos + LEFT + DOWN] in '#~'):
                    #print('added left')
                    self.state[pos + LEFT] = '|'
                    queue.append(pos + LEFT)
            elif self.state[pos + LEFT] not in '|.':
                if leftblock is None:
                    #print(f'blocked left - @{pos + LEFT} is {self.state[pos + LEFT]}')
                    #print(f'blocked left - @{pos + LEFT + DOWN} is {self.state[pos + LEFT + DOWN]}')
                    leftblock = pos
            #print(f'3pos = {pos}')
            if (self.state[pos + RIGHT] == '.') and (self.state[pos + RIGHT + DOWN] in '#~.') and (self.state[pos + DOWN] in '#|~'):
                if (not poppedlast) or (self.state[pos + RIGHT + DOWN] in '#~'):
                    #print('added right')
                    self.state[pos + RIGHT] = '|'
                    queue.append(pos + RIGHT)
            elif self.state[pos + RIGHT] not in '|.':
                if rightblock is None:
                    #print(f'blocked right - @{pos + RIGHT} is {self.state[pos + RIGHT]}')
                    #print(f'blocked right - @{pos + RIGHT + DOWN} is {self.state[pos + RIGHT + DOWN]}')
                    rightblock = pos
            #print(leftblock, ' <> ', rightblock)
            if (leftblock is not None) and (rightblock is not None):
                #print('filled left to right')
                for x in range(int(leftblock.real), int(rightblock.real) + 1):
                    self.state[complex(x, int(leftblock.imag))] = '~'
                leftblock = None
                rightblock = None

    def __str__(self):
        notsand = self.state.keys()
        xmin = min(int(e.real) for e in notsand)
        xmax = max(int(e.real) for e in notsand)
        ymin = min(int(e.imag) for e in notsand)
        ymax = max(int(e.imag) for e in notsand)
        out = []
        out.append((' ' * (500 - xmin)) + '+' + (' ' * (xmax - 500)))
        for y in range(ymin, ymax + 1):
            s = ''
            for x in range(xmin, xmax + 1):
                s += self.state[x + y * 1j]
            out.append(s)
        return '\n'.join(out)
        
    __repr__ = __str__
        

In [73]:
g = GroundScan(testdata)

In [74]:
g

     +      
...........#
#..#.......#
#..#..#.....
#..#..#.....
#.....#.....
#.....#.....
#######.....
............
............
...#.....#..
...#.....#..
...#.....#..
...#######..

In [75]:
g.addWater()
g

     +      
.....|.....#
#..#||||...#
#..#~~#|....
#..#~~#|....
#~~~~~#|....
#~~~~~#|....
#######|....
.......|....
..|||||||||.
..|#~~~~~#|.
..|#~~~~~#|.
..|#~~~~~#|.
..|#######|.

In [76]:
g2 = GroundScan(testdata2)
g2

     +      
...........#
#..#.......#
#..#.##.....
#..#..#.....
#.....#.....
#.....#.....
#######.....
............
............
...#.....#..
...#.....#..
...#.....#..
...#######..

In [77]:
g.addWater()
g

     +      
.....|.....#
#..#||||...#
#..#~##|....
#..#|.#|....
#...|.#|....
#|||||#|....
#######|....
.......|....
..|||||||||.
..|#~~~~~#|.
..|#~~~~~#|.
..|#~~~~~#|.
..|#######|.

Observations
* Once there are two streams checking for bounded left/right fails ('|' vs '~' in the upper container)

In [64]:
c = Counter(g.state.values())

In [65]:
c['|'] + c['~']

57

In [66]:
p1 = GroundScan(data)

In [67]:
p1.addWater()

In [68]:
c = Counter(p1.state.values())

In [69]:
c['|'] + c['~']

200

In [70]:
p1

                                                                                                                                                                                                             +                                                                                                                                       
.............................................................................................................................................................................................................|.......................................................................................................................................
.............................................................................................................................................................................................................|..............................................................................................................